In [21]:
import sqlite3
import pandas as pd 
import telebot
from datetime import datetime

In [67]:
bot = telebot.TeleBot('7136590942:AAGK6n-GZpaQ4WJjtkQjNI83BFuWBxorL84')
privet = "Привет, мой дорогой друг, я твой чат-бот, я буду помогать тебе учиться\nВ случае чего, используй /help"
admins = [761510942]
q1 = 'О каком вебинаре Вы хотите рассказать?'
q2 = 'Что вам больше всего понравилось в теме вебинара и почему?'
q3 = 'Были ли моменты в вебинаре, которые вызвали затруднения в понимании материала? Можете описать их?'
q4 = 'Какие аспекты вебинара, по вашему мнению, нуждаются в улучшении, и какие конкретные изменения Вы бы предложили?'
q5 = 'Есть ли темы или вопросы, которые Вы бы хотели изучить более подробно в следующих занятиях'

@bot.message_handler(commands=['help'])
def help_(message):
    if message.from_user.id in admins:
        help_text = "/start - команда запуска бота с приветственным сообщением\n/sendall - команда отправки рассылки пользователям, которые находятся в базе данных\n/help - список команд"
        bot.send_message(message.from_user.id, help_text)
    else: 
        help_text = "/start - команда запуска бота с приветственным сообщением\n/help - список команд"
        bot.send_message(message.from_user.id, help_text)
        
@bot.message_handler(commands=['start'])
def start(message):
    if message.from_user.id in admins:
        admin_privet = "Привет! Ты зашел с правами администратора, тебе доступно чуть больше команд\nВ случае чего, используй /help"
        bot.send_message(message.from_user.id, admin_privet)
    else:
        bot.send_message(message.from_user.id, privet)
        connect = sqlite3.connect('users.db')
        cursor = connect.cursor()
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS login_id(
        id INT
        )
        """)
        connect.commit()
        
        people = message.chat.id
        cursor.execute(f"SELECT id FROM login_id WHERE id = {people}")
        data = cursor.fetchone()
        if data is None:
            user_id = [message.chat.id]
            cursor.execute("INSERT INTO login_id VALUES(?);", user_id)
            connect.commit()
        

@bot.message_handler(commands=['sendall'])
def sendall(message):
    connect = sqlite3.connect('users.db')
    cursor = connect.cursor()
    
    cursor.execute("SELECT id FROM login_id")
    data = cursor.fetchall()
    
    for i in data:
        text = 'Привет! Оцени свою готовность ответить на пару вопросов от 1 до 10'
        bot.send_message(i[0], text)

        
@bot.message_handler(content_types=['text'])
def first_q(message):
    if message.text in ['5', '6', '7', '8', '9', '10']:
        send = bot.send_message(message.chat.id, q1)
        bot.register_next_step_handler(send, two_q)


def two_q(message):
    global answers
    answers = []
    answers.append(message.from_user.id)
    answers.append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    first_answer = message.text
    answers.append(first_answer)

    send = bot.send_message(message.chat.id, q2)
    bot.register_next_step_handler(send, three_q)


def three_q(message):
    two_answer = message.text
    answers.append(two_answer)

    send = bot.send_message(message.chat.id, q3)
    bot.register_next_step_handler(send, four_q)


def four_q(message):
    three_answer = message.text
    answers.append(three_answer)

    send = bot.send_message(message.chat.id, q4)
    bot.register_next_step_handler(send, five_q)
    
def five_q(message):
    four_answer = message.text
    answers.append(four_answer)

    send = bot.send_message(message.chat.id, q5)
    bot.register_next_step_handler(send, end)


def end(message):
    connect = sqlite3.connect('users.db')
    cursor = connect.cursor()
    
    five_answer = message.text
    answers.append(five_answer)
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS uesrs_answ(
    user_id INT,
    timestamp TEXT,
    question_1 TEXT,
    question_2 TEXT,
    question_3 TEXT,
    question_4 TEXT,
    question_5 TEXT
    )
    """)
    connect.commit()
    
    cursor.execute("INSERT INTO uesrs_answ VALUES(?,?,?,?,?,?,?);", answers)
    connect.commit()
    
    bot.send_message(message.from_user.id, 'Большое спасибо за ответы!')

bot.polling(none_stop=True, interval=0)

In [52]:
cnx = sqlite3.connect('users.db')

df = pd.read_sql_query("SELECT * FROM uesrs_answ", cnx)

In [53]:
df

,user_id,timestamp,question_1,question_2,question_3,question_4,question_5
0,761510942,2024-04-27 21:25:19,sedwqe,ewdwedqwdqw,wdqqwdq,dqwdq,dqwdq
1,673441025,2024-04-27 21:28:35,Хуй,Меня покормили,"Не было, всё понятно",Все ахуенно,Неа
2,673441025,2024-04-27 21:30:06,Зельеваренье,Пиво,Было мало пива,Больше пива,Почему так мало пива?
3,761510942,2024-04-27 21:49:19,dads,dasdas,dasda,asda,dassda
4,1763905530,2024-04-27 21:50:07,Основы программирования,Объяснения преподавателя,Были затруднения в методах словарей,Предложить новые структуры данных к изучению,Визуализация
5,761510942,2024-04-27 21:55:05,das,dsadas,dasdas,dsadas,sadas
6,761510942,2024-04-27 21:58:30,dasdas,asdasd,asda,dada,dada
7,673441025,2024-04-27 22:04:39,Контрольная по эконометрике,После неё мы пошли домой,"Да, говно было есть неудобно","Нужна ложка, руками не очень",Как есть это говно?
8,761510942,2024-04-27 23:57:24,dqwdqw,dqwdqw,dqwdq,dqdq,dqdq
9,853673192,2024-04-28 00:03:51,Проблемы применения машинного обучения для биз...,"Во-первых, это - данные. Недостаток качественн...","Нет, проблем не было",На вебинаре эта проблема была затронута лишь п...,Нет
